In [64]:
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp, mannwhitneyu, spearmanr, shapiro, kstest, lognorm, expon, gamma, cramervonmises

In [65]:
df = pd.read_csv('data_optimized.csv')

In [66]:
df.head()

,author_type,floor,floors_count,rooms_count,total_meters,price_per_month,district,street,underground
0,real_estate_agent,15,20,3,95.0,450000,Пресненский,Мантулинская,Москва-Сити
1,real_estate_agent,35,57,4,209.0,700000,Хорошевский,Чапаевский переулок,Аэропорт
2,real_estate_agent,9,17,1,19.6,36000,Крюково,проспект Георгиевский,Зеленоград — Крюково
3,real_estate_agent,3,8,5,220.0,750000,Хамовники,1-й Зачатьевский переулок,Кропоткинская
4,real_estate_agent,2,29,1,29.0,27500,Хорошево-Мневники,Шелепихинская набережная,Шелепиха


### Просмотр графиков, конечно же, натолкнул меня на некие _гипотезы_, и сейчас мы рассмотрим их 😇  
p.s. у меня ещё не так сильна база по матстату, поэтому гипотезы не самые сложные, но имеем что имеем..

#### 1. Влияние первого этажа на цену

Посмотрим, имеют ли обе выборки нормальное распределение.

In [67]:
first_floor_prices = df[df['floor'] == 1]['price_per_month']
other_floors_prices = df[df['floor'] != 1]['price_per_month']
ks_2samp(first_floor_prices, other_floors_prices)

KstestResult(statistic=np.float64(0.22443762781186094), pvalue=np.float64(0.04945175988220721), statistic_location=np.int64(78200), statistic_sign=np.int8(1))

_p-value_ оказался меньше 0.05, что говорит о том, что данные НЕ имеют нормальное распределение. Значит, будем использовать не z/t тесты, а что-нибудь другое: например, U-тест Манна-Уитни.  
  
$H_0$: квартиры на 1 этаже стоят в среднем __не__ дешевле, чем квартиры на всех остальных этажах  
$H_1$: квартиры на 1 этаже стоят в среднем дешевле, чем квартиры на всех остальных этажах

In [68]:
statistic, p_value = mannwhitneyu(first_floor_prices, other_floors_prices, alternative='less')
p_value

np.float64(0.040480091285698676)

_p_value_ меньше 0.05, а значит, мы отвергаем нашу $H_0$ гипотезу — квартиры на первом этаже в среднем стоят дешевле, чем квартиры на остальных этажах.

---

#### 2. Влияние последнего этажа на цену

Посмотрим, имеют ли обе выборки нормальное распределение.

In [69]:
last_floor_prices = df[df['floor'] == df['floors_count']]['price_per_month']
middle_floor_prices = df[(df['floor'] != 1) & (df['floor'] != df['floors_count'])]['price_per_month']
ks_2samp(last_floor_prices, middle_floor_prices)

KstestResult(statistic=np.float64(0.1486126594822247), pvalue=np.float64(0.022395144799830454), statistic_location=np.int64(349900), statistic_sign=np.int8(-1))

_p_value_ оказался меньше 0.05, значит, будем использовать U-тест Манна-Уитни.  
  
$H_0$: квартиры на последнем этаже стоят в среднем __не__ дороже, чем квартиры на средних этажах  
$H_1$: квартиры на последнем этаже стоят в среднем дороже, чем квартиры на средних этажах


In [70]:
statistic, p_value = mannwhitneyu(last_floor_prices, middle_floor_prices, alternative='greater')
p_value

np.float64(0.0037312323833731814)

_p_value_ меньше 0.05, значит, мы отвергаем $H_0$ гипотезу — квартиры на последних этажах стоят в среднем дороже, чем квартиры на средних этажах.  

---

#### 3. Корреляция между площадью квартиры и ценой

Посмотрим, имеют ли обе выборки нормальное распределение.

In [71]:
ks_2samp(df['total_meters'], df['price_per_month'])

KstestResult(statistic=np.float64(1.0), pvalue=np.float64(0.0), statistic_location=np.float64(455.0), statistic_sign=np.int8(1))

_p_value_ = 0, значит, выборки снова максимально НЕ распределены нормально, снова не z/t тест 💔😗  
Будем использовать непараметрический метод для анализа корреляции — коэффициент корреляции Спирмена.  

$H_0$: __не__ наблюдается значимой корреляции между площадью квартиры и ценой  
$H_1$: наблюдается значимая корреляция между площадью квартиры и ценой

In [72]:
corr, p_value = spearmanr(df['total_meters'], df['price_per_month'])
corr, p_value

(np.float64(0.8651369901446254), np.float64(0.0))

_p_value_ меньше 0.05, значит, мы отвергаем $H_0$ — значимая корреляция присутствует, причём она равна 0.86, что соответствует довольно сильной положительной корреляции.

---

#### 4. Распределение площади

$H_0$: данные площади распределены нормально  
$H_1$: данные площади распределены __не__ нормально

In [73]:
shapiro_stat, shapiro_p = shapiro(df['total_meters']) # тест Шапиро-Уилка на нормальность, для разнообразия
shapiro_p

np.float64(3.5339591739175074e-37)

Число ооочень малое, значит, отвергаем $H_0$: данные распределены однозначно __не__ нормально 😃  
Давайте тогда исследуем, может, данные похожи на какое-то другое распределение:

In [74]:
params_lognorm = lognorm.fit(df['total_meters'])
ks_stat_lognorm, ks_p_lognorm = kstest(df['total_meters'], 'lognorm', args=params_lognorm)
ks_p_lognorm # < 0.05, не логнормальное распределение(

np.float64(0.0012037589206601326)

In [75]:
params_expon = expon.fit(df['total_meters'])
ks_stat_expon, ks_p_expon = kstest(df['total_meters'], 'expon', args=params_expon)
ks_p_expon # < 0.05, не экспоненциальное распределение((

np.float64(2.8762527185323475e-23)

In [76]:
params_gamma = gamma.fit(df['total_meters']) # тест Крамера-фон Мизеса 😵 на гамма-распределение!
cvm_result = cramervonmises(df['total_meters'], 'gamma', args=params_gamma)
cvm_result.pvalue # < 0.05, не гамма распределение(((

np.float64(1.4739258147322687e-07)

По итогу, ближе всего распределение данных оказалось к логнормальному, если снизим нашу планку по alpha до 0.01 😊

---

#### 5. Влияние этажности на цену

Как обычно, проверим нормальность данных:

In [77]:
ks_2samp(df['floors_count'], df['price_per_month'])

KstestResult(statistic=np.float64(1.0), pvalue=np.float64(0.0), statistic_location=np.int64(95), statistic_sign=np.int8(1))

Вообще не нормально! Идём к нашей корреляции Спирмена:  

$H_0$: __не__ наблюдается значимой корреляции между этажостью квартиры и ценой  
$H_1$: наблюдается значимая корреляция между этажностью квартиры и ценой

In [78]:
corr, p_value = spearmanr(df['floors_count'], df['price_per_month'])
corr, p_value

(np.float64(-0.09056114146006751), np.float64(0.000904019937556351))

_p_value_ меньше 0.05, значит, мы отвергаем $H_0$: наблюдается значимая корреляция между этажностью и ценой = -0.09, хоть она и не особо сильная.

---

#### 6. Выбросы

$H_0$: в данных меньше 50 выбросов  
$H_1$: в данных больше 50 выбросов

Посчитаем количество выбросов методом интерквартильного размаха: будем считать выбросами те значения, что отклоняются от квартилей более чем на 1.5 интерквартильных размахов

In [79]:
Q1 = np.percentile(df['price_per_month'], 25) # считаем наши квартили
Q3 = np.percentile(df['price_per_month'], 75)
down = Q1 - 1.5 * (Q3 - Q1) # считаем наши границы: квартиль +/- 1.5 интерквартильного размаха
up  =  Q3 + 1.5 * (Q3 - Q1)

out = df[(df['price_per_month'] < down) | (df['price_per_month'] > up)] # сами выбросы (если попадают в одну из границ)

print(f"Количество выбросов: {len(out)}")

Количество выбросов: 83


Больше 50 выбросов, отвергаем $H_0$ 😕  
Но это всё ещё хороший результат, т.к. всего у нас 1340 наблюдений, а значит, выбросы составляют всего 6.19% наблюдений!  
При увеличении выборки, количество выбросов скорее всего будет иметь тенденцию к уменьшению (т.к. интерквартильных размах может увеличиться)